# Cálculo de 

In [158]:
import pandas as pd
import numpy as np
from itertools import pairwise

In [159]:
np.set_printoptions(precision=3, floatmode='fixed', suppress=True)
pd.options.display.precision = 3
dtype = np.float64

In [160]:
headers = np.dtype( [('r', dtype), ('alpha', dtype)] )
data = np.array(
    [
        (100., np.deg2rad(30)),
        ( 90., np.deg2rad(40)),
        (110., np.deg2rad(20)),
    ],
    # dtype=headers
)
print(data)

[[100.000   0.524]
 [ 90.000   0.698]
 [110.000   0.349]]


In [161]:
def st(array):
    if isinstance(array, np.ndarray):
        assert array.shape == (2,), 'array shape must be (2,)'
    else:
        assert len(array) == 2
    result = array[0] * np.tan( array[1]/2 )
    return result
st_vec = lambda matrix: np.apply_along_axis(st, 1, matrix)
st_vec(data)

array([26.795, 32.757, 19.396])

In [162]:
np.sum(data, axis=0)

array([300.000,   1.571])

In [163]:
def diag_(array):
    assert array.shape == (2,2), "Array shape must be 2 x 2."
    sum_ = np.sum(array, axis=0)
    angle = (np.pi) - sum_[1]
    side = sum_[0]
    return np.array(( (side * np.sin(array[0,1]) / np.sin(angle), sum_[1])))

In [164]:
test = np.array([
    (25, np.deg2rad(13)),
    ( 5, np.deg2rad(6)),
    (15, np.deg2rad(17)),
    (10, np.deg2rad(9))
])

In [165]:
diag_(test[::-1][:2])

array([8.921, 0.454])

In [166]:
# def diag(array, inverse = True):
#     llist = []
#     if inverse: array = array[::-1]
#     for pair in pairwise(array):
#         pair_a = np.array(pair)
#         vec_ = st_vec(pair_a)
#         sum_ = np.sum(vec_)
#         sum_a = np.sin(np.sum(np.array(pair_a), axis=0)[1])
#         llist.append((sum_ * np.sin(pair_a[0,1]) / sum_a))
#     return np.array(llist)
# diag(data)

In [167]:
st_vec(data)

array([26.795, 32.757, 19.396])

In [168]:
import copy
def diag(array, st_= False):
    llist = []
    array_ = copy.deepcopy(array)
    if st_:
        array_[:,0] = st_vec(array)
    # print(array_)
    for i in range(array_.shape[0] - 1):
        op = array_[i:i+2,:]
        # print("->", op)
        diag__ = diag_(op)
        array_[i+1,0] += diag__[0]
        array_[i+1,1] = diag__[1]
        # print("==>\n", array_)
        # print("=>",op[i])
        # print("->",diag__)
        llist.append(diag__)
    return np.array(llist)

In [169]:
diag(data[::-1], st_=True)[-1,0]

69.41122834116955

In [170]:
print(data[:,0])
print(st_vec(data))

[100.000  90.000 110.000]
[26.795 32.757 19.396]


In [171]:
st_vec(data)

array([26.795, 32.757, 19.396])

In [172]:
def pc_calc(data, pi=0):
    acc_inv = st_vec(data)[0]
    acc_inv += diag(data[::-1], st_=True)[-1,0]
    acc_inv = pi - acc_inv
    return acc_inv
pc_calc(data, pi=14_000)

13903.793852415718

In [173]:
def p_pc(data, pi=0):
    pc_ = pc_calc(data, pi)
    prod_ = np.prod(data, axis=1)
    acc_ = np.add.accumulate(prod_)
    return np.insert(acc_ + pc_, 0, pc_, axis=0)
p_pc(data, pi=14_000).reshape(-1,1)

array([[13903.794],
       [13956.154],
       [14018.986],
       [14057.383]])

In [174]:
test = np.array(
    [
        (10,np.deg2rad(8.611)),
        (15,np.deg2rad(17.408)),
        ( 5,np.deg2rad(5.591)),
        (25,np.deg2rad(13.390))
    ]
)

In [175]:
diag(test[::-1])

array([[21.360,  0.331],
       [22.675,  0.635],
       [40.000,  0.785]])

In [176]:
ZERO_PAR = np.array((0,0)).reshape(1,-1)
_ = np.concatenate((data, ZERO_PAR), axis=0)
_ = np.concatenate((_, p_pc(data, pi=14_000).reshape(-1,1)), axis=1)
_ = pd.DataFrame(_, columns=['radios', 'angulos', 'cota_ac'])
_.style.format({'radios': '{:} m', 'angulos': lambda x: round(np.rad2deg(x),3), 'cota_ac': '{:,.3f}'} ,precision=3)
# print(pc_calc(data, pi=14_000))

,radios,angulos,cota_ac
0,100.0 m,30.0,"13,903.794"
1,90.0 m,40.0,"13,956.154"
2,110.0 m,20.0,"14,018.986"
3,0.0 m,0.0,"14,057.383"


In [230]:
def gen_marks(array, prec = 1):
    step = 10 ** prec
    start = ((array[0] // 10**prec) + 1) * step
    end = ((array[-1] // 10**prec) + 1) * step
    arange = np.arange(start, end, step,)
    return np.unique(np.sort(np.concatenate([arange, array])), axis=0)
def diff_marks(array):
    return np.append( (0), np.ediff1d(array))
    # assert marks.ndim == 1, "marks argument must be 1-Dimensional"
    # llist = [0]
    # for low, sup in pairwise(marks):
    #     llist.append(sup - low)
    # return np.array(llist)

In [220]:
cotas = p_pc(data, pi = 14_000)
cotas

In [207]:
#acc_marks
marks = gen_marks(cotas)
marks

array([13903.794, 13910.000, 13920.000, 13930.000, 13940.000, 13950.000,
       13956.154, 13960.000, 13970.000, 13980.000, 13990.000, 14000.000,
       14010.000, 14018.986, 14020.000, 14030.000, 14040.000, 14050.000,
       14057.383])

In [222]:
# diff_marks
diff_marks(marks)

array([ 0.000,  6.206, 10.000, 10.000, 10.000, 10.000,  6.154,  3.846,
       10.000, 10.000, 10.000, 10.000, 10.000,  8.986,  1.014, 10.000,
       10.000, 10.000,  7.383])

In [211]:
np.add.accumulate(diff_marks(marks))

array([  0.000,   6.206,  16.206,  26.206,  36.206,  46.206,  52.360,
        56.206,  66.206,  76.206,  86.206,  96.206, 106.206, 115.192,
       116.206, 126.206, 136.206, 146.206, 153.589])

In [306]:
def expand_idx(cotas, prec=1):
    return np.array(np.round(np.ediff1d(cotas), -prec) // 10**prec, dtype=np.uint) + 1

In [428]:
def expand__(matrix, prec=1, pi=0):
    matrix = copy.deepcopy(matrix)
    pc = p_pc(matrix, pi)
    marks = gen_marks(pc, prec=prec)
    pc = pc[1:]
    # matrix = np.concatenate((matrix, ZERO_PAR))
    matrix = np.concatenate((matrix, pc.reshape(-1,1)), axis=1)
    llist = []
    iiter = iter(matrix)
    vector = next(iiter)
    for mark in marks[:-1]:
        if mark == vector[2]:
            vector = next(iiter)
            # continue
        llist.append( np.append(vector[:2], mark))
    return np.append( np.array(llist), ((0,0,marks[-1]),), axis=0)
        
        

In [429]:
expand__(data, pi=14_000)

array([[  100.000,     0.524, 13903.794],
       [  100.000,     0.524, 13910.000],
       [  100.000,     0.524, 13920.000],
       [  100.000,     0.524, 13930.000],
       [  100.000,     0.524, 13940.000],
       [  100.000,     0.524, 13950.000],
       [   90.000,     0.698, 13956.154],
       [   90.000,     0.698, 13960.000],
       [   90.000,     0.698, 13970.000],
       [   90.000,     0.698, 13980.000],
       [   90.000,     0.698, 13990.000],
       [   90.000,     0.698, 14000.000],
       [   90.000,     0.698, 14010.000],
       [  110.000,     0.349, 14018.986],
       [  110.000,     0.349, 14020.000],
       [  110.000,     0.349, 14030.000],
       [  110.000,     0.349, 14040.000],
       [  110.000,     0.349, 14050.000],
       [    0.000,     0.000, 14057.383]])

In [435]:
def expand(matrix, prec=1, pi=0):
    expanded = expand__(matrix, prec=prec, pi=pi)
    return np.append( expanded, diff_marks(expanded[:,2]).reshape(-1,1),axis=1)

In [436]:
expand(data, pi=14_000)

array([[  100.000,     0.524, 13903.794,     0.000],
       [  100.000,     0.524, 13910.000,     6.206],
       [  100.000,     0.524, 13920.000,    10.000],
       [  100.000,     0.524, 13930.000,    10.000],
       [  100.000,     0.524, 13940.000,    10.000],
       [  100.000,     0.524, 13950.000,    10.000],
       [   90.000,     0.698, 13956.154,     6.154],
       [   90.000,     0.698, 13960.000,     3.846],
       [   90.000,     0.698, 13970.000,    10.000],
       [   90.000,     0.698, 13980.000,    10.000],
       [   90.000,     0.698, 13990.000,    10.000],
       [   90.000,     0.698, 14000.000,    10.000],
       [   90.000,     0.698, 14010.000,    10.000],
       [  110.000,     0.349, 14018.986,     8.986],
       [  110.000,     0.349, 14020.000,     1.014],
       [  110.000,     0.349, 14030.000,    10.000],
       [  110.000,     0.349, 14040.000,    10.000],
       [  110.000,     0.349, 14050.000,    10.000],
       [    0.000,     0.000, 14057.383,     7

In [449]:
def gamma_rad(array):
    "Function that calculates gamma value. Angle must be in radians."
    assert array.shape[0] == 2
    if array[0] == 0:
        return np.float_(0)
    return array[1] / (2 * array[0])
gamma_rad_vec = lambda matrix: np.apply_along_axis(gamma_rad, 1, matrix)

In [462]:
np.set_printoptions(precision=10)

In [498]:
def deg_to_deg(array):
    llist = []
    for value in array:
        int_ = int(value)
        b = (value - int_) * 60
        c = (b - int(b)) * 60
        llist.append('{}º {}" {}`'.format(int_, int(b), round(float(c),3)))
    return np.array(llist)

In [483]:
int(3.14)

3

In [499]:
_ = expand(data, pi=14_000)
_ = gamma_rad_vec(_[:,[0,3]])
_ = np.rad2deg(_.reshape(-1,1))
print(deg_to_deg(_).reshape(-1,1))

[['0º 0" 0.0`']
 ['1º 46" 40.549`']
 ['2º 51" 53.24`']
 ['2º 51" 53.24`']
 ['2º 51" 53.24`']
 ['2º 51" 53.24`']
 ['1º 57" 31.655`']
 ['1º 13" 27.501`']
 ['3º 10" 59.156`']
 ['3º 10" 59.156`']
 ['3º 10" 59.156`']
 ['3º 10" 59.156`']
 ['3º 10" 59.156`']
 ['2º 20" 24.589`']
 ['0º 15" 51.084`']
 ['2º 36" 15.673`']
 ['2º 36" 15.673`']
 ['2º 36" 15.673`']
 ['0º 0" 0.0`']]


In [501]:
deg_to_deg(np.add.accumulate(_)).reshape(-1,1)

array([['0º 0" 0.0`'],
       ['1º 46" 40.549`'],
       ['4º 38" 33.789`'],
       ['7º 30" 27.03`'],
       ['10º 22" 20.27`'],
       ['13º 14" 13.51`'],
       ['15º 11" 45.166`'],
       ['16º 25" 12.666`'],
       ['19º 36" 11.822`'],
       ['22º 47" 10.978`'],
       ['25º 58" 10.134`'],
       ['29º 9" 9.29`'],
       ['32º 20" 8.446`'],
       ['34º 40" 33.035`'],
       ['34º 56" 24.119`'],
       ['37º 32" 39.792`'],
       ['40º 8" 55.465`'],
       ['42º 45" 11.138`'],
       ['42º 45" 11.138`']], dtype='<U15')

In [317]:
aaa = iter(data)
print(next(aaa))
print(next(aaa))

[100.000   0.524]
[90.000  0.698]


In [287]:
gen_marks(p_pc(data, pi=14_000), prec=0).__len__()

158

In [293]:
def expand(data, prec=1, pi=0):
    data = copy.deepcopy(data)
    match data.shape[1]:
        case 2:
            pc = p_pc(data, pi=pi)
        case 3:
            pc = data[:,2]
            data = data[:,:2]
        case _:
            raise Exception
    idx = expand_idx(pc, prec=prec)
    print(sum(idx))
    # if prec == 0:
    repeated = np.repeat(data, idx, axis=0)
    repeated = np.append(repeated, ZERO_PAR, axis=0)
    print(repeated.shape)
    marks = gen_marks(pc, prec).reshape(-1,1)
    print(marks.shape)
    diff_marks_ = diff_marks(marks).reshape(-1,1)
    print(diff_marks_.shape)
    return np.concatenate((repeated, marks, diff_marks_), axis=1)
    # print(data)
    # print(pc)
    # dims = p_pc(data)
    # marks = mark_(, prec)
    # print(repeated)
    # print(pc)
    # data = np.append(data, (0,0))
    # return np.repeat(data, np.append(idx_, (1)), axis=0)
    # return data

In [307]:
expand(data, pi=14_000, prec=1)

18
(19, 2)
(19, 1)
(19, 1)


array([[  100.000,     0.524, 13903.794,     0.000],
       [  100.000,     0.524, 13910.000,     6.206],
       [  100.000,     0.524, 13920.000,    10.000],
       [  100.000,     0.524, 13930.000,    10.000],
       [  100.000,     0.524, 13940.000,    10.000],
       [  100.000,     0.524, 13950.000,    10.000],
       [   90.000,     0.698, 13956.154,     6.154],
       [   90.000,     0.698, 13960.000,     3.846],
       [   90.000,     0.698, 13970.000,    10.000],
       [   90.000,     0.698, 13980.000,    10.000],
       [   90.000,     0.698, 13990.000,    10.000],
       [   90.000,     0.698, 14000.000,    10.000],
       [   90.000,     0.698, 14010.000,    10.000],
       [  110.000,     0.349, 14018.986,     8.986],
       [  110.000,     0.349, 14020.000,     1.014],
       [  110.000,     0.349, 14030.000,    10.000],
       [  110.000,     0.349, 14040.000,    10.000],
       [  110.000,     0.349, 14050.000,    10.000],
       [    0.000,     0.000, 14057.383,     7

In [ ]:
def gamma()

In [114]:
data

array([[100.000,   0.524],
       [ 90.000,   0.698],
       [110.000,   0.349]])

In [117]:
expand(data).reshape(-1,2)

array([[100.000,   0.524],
       [ 90.000,   0.698],
       [110.000,   0.349],
       [  0.000,   0.000]])

In [100]:
expand(cotas).reshape(-1,1)

array([[13903.794],
       [13956.154]])

In [79]:
_ = np.array(list(pairwise(np.round(cotas, -1))))
np.array(np.round(np.diff(_, axis=1), -1) // 10, dtype=np.int_) + 1

array([[7],
       [7],
       [5]])

In [67]:
expand(cotas)

array([6, 7, 4])

In [47]:
np.repeat(cotas, [1,2,3,4], axis=0).reshape(-1,1)

array([[13903.794],
       [13956.154],
       [13956.154],
       [14018.986],
       [14018.986],
       [14018.986],
       [14057.383],
       [14057.383],
       [14057.383],
       [14057.383]])